<a href="https://colab.research.google.com/github/PY1807/Machine-Learning-Projects/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras import layers
from keras.models import Model
from keras.layers import SimpleRNN, LSTM, Bidirectional


from keras.layers import Embedding, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam



In [ ]:
df=pd.read_csv('test.csv', encoding = "cp1252")

In [ ]:
df.head()

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [ ]:
data=pd.DataFrame(df,columns=['text','sentiment'])

In [ ]:
data

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
4810,NaN,NaN
4811,NaN,NaN
4812,NaN,NaN
4813,NaN,NaN


In [ ]:
data.isna().sum()

text         1281
sentiment    1281
dtype: int64

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.shape

(3534, 2)

In [ ]:
data.duplicated().sum()

0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3534 entries, 0 to 3533
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3534 non-null   object
 1   sentiment  3534 non-null   object
dtypes: object(2)
memory usage: 82.8+ KB


In [ ]:
data['sentiment'].value_counts()

sentiment
neutral     1430
positive    1103
negative    1001
Name: count, dtype: int64

In [ ]:
data.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [ ]:
for i in range(10):
  print(data['text'][i+1])

 Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China:  (SH)  (BJ).
Recession hit Veronique Branquinho, she has to quit her company, such a shame!
 happy bday!
 http://twitpic.com/4w75p - I like it!!
 that`s great!! weee!! visitors!
I THINK EVERYONE HATES ME ON HERE   lol
 soooooo wish i could, but im in school and myspace is completely blocked
 and within a short time of the last clue all of them
 What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!
My bike was put on hold...should have known that.... argh total bummer


In [ ]:
def clean(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    # Conver to lower
    text = text.lower()
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
data['text'] = data['text'].apply(clean)

In [ ]:
for i in range(10):
  print(data['text'][i+1])

shanghai is also really exciting precisely skyscrapers galore good tweeps in china sh bj
recession hit veronique branquinho she has to quit her company such a shame
happy bday
httptwitpiccomwp i like it
thats great weee visitors
i think everyone hates me on here lol
soooooo wish i could but im in school and myspace is completely blocked
and within a short time of the last clue all of them
what did you get my day is alright havent done anything yet leaving soon to my stepsister though
my bike was put on holdshould have known that argh total bummer


In [ ]:
# Tokenize Text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])

In [ ]:
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen = max_length, padding = 'post')

In [ ]:
labels = pd.get_dummies(data['sentiment']).values

In [ ]:
labels

array([[False,  True, False],
       [False, False,  True],
       [ True, False, False],
       ...,
       [ True, False, False],
       [False, False,  True],
       [False, False,  True]])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(padded_sequences, labels, test_size = 0.33)

In [ ]:
xtrain

array([[   1,   45,    1, ...,    0,    0,    0],
       [ 612, 4677,    0, ...,    0,    0,    0],
       [  21, 1059,  327, ...,    0,    0,    0],
       ...,
       [ 524,    7, 2750, ...,    0,    0,    0],
       [ 295,   72, 1562, ...,    0,    0,    0],
       [ 100,  536,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
xtest

array([[ 136, 1328,  185, ...,    0,    0,    0],
       [ 678,    1,   71, ...,    0,    0,    0],
       [  28,   72, 3093, ...,    0,    0,    0],
       ...,
       [  67,    7, 2101, ...,    0,    0,    0],
       [1173,  246,  982, ...,    0,    0,    0],
       [ 406,   58,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
ytrain

array([[False, False,  True],
       [ True, False, False],
       [False,  True, False],
       ...,
       [False, False,  True],
       [False,  True, False],
       [False, False,  True]])

In [ ]:
ytest

array([[False,  True, False],
       [ True, False, False],
       [False, False,  True],
       ...,
       [ True, False, False],
       [ True, False, False],
       [False,  True, False]])

Model Buliding


In [ ]:
def create_model(model_type, vocab_size, embedding_dim, num_classes=3):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))

    if model_type == 'RNN':
        model.add(SimpleRNN(32, return_sequences=True))
    elif model_type == 'LSTM':
        model.add(LSTM(32, return_sequences=True))
    elif model_type == 'BiLSTM':
        model.add(Bidirectional(LSTM(32, return_sequences=True)))
    else:
        raise ValueError("Invalid model type. Choose from 'RNN', 'LSTM', 'BiLSTM'.")

    model.add(BatchNormalization())

    if model_type == 'RNN':
        model.add(SimpleRNN(64))
    elif model_type == 'LSTM':
        model.add(LSTM(64))
    elif model_type == 'BiLSTM':
        model.add(Bidirectional(LSTM(64)))

    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu'))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model


In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
num_classes = 3
for i in range(3,14):
  print(f"Training RNN model...")
  model1 = create_model('RNN', vocab_size, embedding_dim, num_classes)
  model1.fit(xtrain, ytrain, epochs=i, validation_data=(xtest, ytest))

  loss, accuracy = model1.evaluate(xtest, ytest)
  print("Test Accuracy:", accuracy)

Training RNN model...
Epoch 1/3
74/74 [==============================] - 7s 42ms/step - loss: 1.1817 - accuracy: 0.3540 - val_loss: 1.0967 - val_accuracy: 0.3899
Epoch 2/3
74/74 [==============================] - 4s 54ms/step - loss: 0.9700 - accuracy: 0.5239 - val_loss: 1.1316 - val_accuracy: 0.3359
Epoch 3/3
37/37 [==============================] - 0s 7ms/step - loss: 1.1956 - accuracy: 0.4036
Test Accuracy: 0.40359896421432495
Training RNN model...
Epoch 1/4
74/74 [==============================] - 6s 43ms/step - loss: 1.1529 - accuracy: 0.3676 - val_loss: 1.0977 - val_accuracy: 0.4096
Epoch 2/4
74/74 [==============================] - 4s 56ms/step - loss: 0.9941 - accuracy: 0.5061 - val_loss: 1.1519 - val_accuracy: 0.3059
Epoch 3/4
74/74 [==============================] - 3s 37ms/step - loss: 0.5638 - accuracy: 0.7812 - val_loss: 1.2664 - val_accuracy: 0.3685
Epoch 4/4
37/37 [==============================] - 0s 7ms/step - loss: 1.4576 - accuracy: 0.4122
Test Accuracy: 0.4121679663

In [ ]:
for i in range(3,14):
  print(f"Training LSTM model...")
  model2 = create_model('LSTM', vocab_size, embedding_dim, num_classes)
  model2.fit(xtrain, ytrain, epochs=i, validation_data=(xtest, ytest))

  loss, accuracy = model2.evaluate(xtest, ytest)
  print("Test Accuracy:", accuracy)

Training LSTM model...
Epoch 1/3
74/74 [==============================] - 11s 77ms/step - loss: 1.1014 - accuracy: 0.4161 - val_loss: 1.0988 - val_accuracy: 0.2836
Epoch 2/3
74/74 [==============================] - 5s 64ms/step - loss: 0.7491 - accuracy: 0.6688 - val_loss: 1.0937 - val_accuracy: 0.2656
Epoch 3/3
37/37 [==============================] - 0s 12ms/step - loss: 1.0321 - accuracy: 0.5056
Test Accuracy: 0.5055698156356812
Training LSTM model...
Epoch 1/4
74/74 [==============================] - 11s 68ms/step - loss: 1.1127 - accuracy: 0.4060 - val_loss: 1.0954 - val_accuracy: 0.4096
Epoch 2/4
74/74 [==============================] - 4s 54ms/step - loss: 0.7440 - accuracy: 0.6671 - val_loss: 1.0809 - val_accuracy: 0.4096
Epoch 3/4
74/74 [==============================] - 5s 75ms/step - loss: 0.3943 - accuracy: 0.8559 - val_loss: 1.0355 - val_accuracy: 0.5304
Epoch 4/4
37/37 [==============================] - 0s 12ms/step - loss: 0.9851 - accuracy: 0.5141
Test Accuracy: 0.51413

In [ ]:
for i in range(3,14):
  print(f"Training BiLSTM model...")
  model2 = create_model('BiLSTM', vocab_size, embedding_dim, num_classes)
  model2.fit(xtrain, ytrain, epochs=i, validation_data=(xtest, ytest))

  loss, accuracy = model2.evaluate(xtest, ytest)
  print("Test Accuracy:", accuracy)

Training BiLSTM model...
Epoch 1/3
74/74 [==============================] - 19s 134ms/step - loss: 1.0406 - accuracy: 0.4639 - val_loss: 1.1155 - val_accuracy: 0.2836
Epoch 2/3
74/74 [==============================] - 6s 79ms/step - loss: 0.4763 - accuracy: 0.8150 - val_loss: 1.1088 - val_accuracy: 0.4165
Epoch 3/3
37/37 [==============================] - 1s 17ms/step - loss: 1.1221 - accuracy: 0.3933
Test Accuracy: 0.39331620931625366
Training BiLSTM model...
Epoch 1/4
74/74 [==============================] - 19s 135ms/step - loss: 1.0580 - accuracy: 0.4778 - val_loss: 1.0884 - val_accuracy: 0.4182
Epoch 2/4
74/74 [==============================] - 6s 83ms/step - loss: 0.4398 - accuracy: 0.8344 - val_loss: 1.0753 - val_accuracy: 0.4096
Epoch 3/4
74/74 [==============================] - 8s 104ms/step - loss: 0.1579 - accuracy: 0.9480 - val_loss: 1.0554 - val_accuracy: 0.4473
Epoch 4/4
37/37 [==============================] - 1s 18ms/step - loss: 1.0148 - accuracy: 0.5030
Test Accuracy:

By Observation we get to know that BiLSTM Model with 6 epochs has the highest accuracy-57.41% among all possibilities.

In [ ]:
print(f"Training BiLSTM model...")
model = create_model('BiLSTM', vocab_size, embedding_dim, num_classes)
model.fit(xtrain, ytrain, epochs=6, validation_data=(xtest, ytest))

loss, accuracy = model.evaluate(xtest, ytest)
print("Test Accuracy:", accuracy)


Training BiLSTM model...
Epoch 1/6
74/74 [==============================] - 17s 103ms/step - loss: 1.0791 - accuracy: 0.4444 - val_loss: 1.0842 - val_accuracy: 0.4096
Epoch 2/6
74/74 [==============================] - 7s 101ms/step - loss: 0.5015 - accuracy: 0.8048 - val_loss: 1.1078 - val_accuracy: 0.4096
Epoch 3/6
74/74 [==============================] - 6s 81ms/step - loss: 0.1559 - accuracy: 0.9552 - val_loss: 1.0874 - val_accuracy: 0.4293
Epoch 4/6
74/74 [==============================] - 7s 101ms/step - loss: 0.0548 - accuracy: 0.9861 - val_loss: 1.0639 - val_accuracy: 0.4473
Epoch 5/6
74/74 [==============================] - 6s 80ms/step - loss: 0.0395 - accuracy: 0.9894 - val_loss: 1.1233 - val_accuracy: 0.4062
Epoch 6/6
37/37 [==============================] - 1s 16ms/step - loss: 1.0221 - accuracy: 0.5167
Test Accuracy: 0.5167095065116882


In [ ]:
def predict_sentiment(input_text, tokenizer, model, max_length):
    # Preprocess the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length, padding='post')

    # Get the prediction
    prediction = model.predict(padded_input_sequence)

    # Convert the prediction to sentiment label
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    predicted_label_index = np.argmax(prediction)
    predicted_sentiment = sentiment_labels[predicted_label_index]

    return predicted_sentiment

In [ ]:
input_text = "Beautiful!"
predicted_sentiment = predict_sentiment(input_text, tokenizer, model, max_length)
print("Predicted Sentiment:", predicted_sentiment)

1/1 [==============================] - 2s 2s/step
Predicted Sentiment: Neutral
